In [115]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy, deepcopy
from tqdm.notebook import tqdm


import numpy as np

In [2]:
PROBLEM_SIZE = 1_000
NUM_SETS = 5_000
SETS = tuple(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [77]:
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness3(state):
    return sum(state)

fitness = fitness3

In [4]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

## Single-State Methods

They are used to find local best solutions.

### General procedure
- **Initialization**: choose a starting candidate solution
- **Evaluation**: evaluate the quality of the current solution using the `objective or fitness_function`
- **Modification**: `tweak` the current solution to obtain a new candidate solution (modify it a little bit)
- **Selection**: choose which candidate solution to keep
- **Termination**: stop when a termination criterion is met
  - Maximum number of iterations
  - Maximum number of evaluations
  - Wall-clock time
  - Quality threshold

#### Tweaking

Optimization algorithms define specific `tweak operators` tailored to the problem. These operators dictate how to modify solutions, such as:
- swapping elements
- adjusting values
- applying random noise

🔎 **Exploitation**: make small adjustments to the current solution to improve it
1. *Adjust the modification procedure*, usually making a `small tweak` to the current solution
2. *Use a larger sample*, exploring a larger region of the search space (`neighborhood`) at each step
  
🗺️ **Exploration**: make large adjustments to explore a different region of the search space

1. *Adjust the modification procedure*, occasionally making a `big tweak` to explore a different region of the search space
2. *Adjust the selection procedure*, occasionally accept a `worse solution` to avoid getting stuck in a local optimum
3. *Jump to sth new*, restarting the algorithm from a `different random starting point` to explore a different region of the search space

## Hill-Climbing

* `Random Mutation HC`: It just iteratively test new candidate solutions in the region of the current candidate, and adopt the new ones if they're better.
* `Steepest ascent HC`: it generates multiple "tweaks" to a candidate solution simultaneously, and then it selects the best-performing one for adoption.
* `Random-restart HC`: In order to avoid getting stuck in a local optimum, we can restart the algorithm from a different random starting point. Note that we're just increasing the probability of finding the global optimum, but we're not guaranteed to find it.

In [136]:
def random_mutation_hc(problem, max_iterations):
    number_iterations = 0

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1
        actual_state = deepcopy(problem)
        next_state = tweak(problem)

        if(fitness(next_state) > fitness(actual_state)):
            problem = next_state

            # If we have reached a satisfactory solution, stop
            if(fitness(problem) == PROBLEM_SIZE):
                break

    return problem, number_iterations

def steepest_ascend_hc(problem, max_iterations, neighbours_size):
    number_iterations = 0

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1
        actual_state = deepcopy(problem)
        
        # Get all the neighbours
        neighbours = [tweak(problem) for _ in range(neighbours_size)]
        best_neighbour = max(neighbours, key=fitness)
        if(fitness(best_neighbour) > fitness(actual_state)):
            problem = best_neighbour

            # If we have reached a satisfactory solution, stop
            if(fitness(problem) == PROBLEM_SIZE):
                break

    return problem, number_iterations

def random_restart_hc(max_iterations, restarts):
    best_solution = None
    number_of_restarts = 0

    # Loop until we reach the maximum number or restarts
    for _ in range(restarts):
        problem = np.array([choice([True, False, False, False, False]) for _ in range(PROBLEM_SIZE)])
        number_of_restarts += 1
        problem, _ = random_mutation_hc(problem, max_iterations)

        if(best_solution is None or fitness(problem) > fitness(best_solution)):
            best_solution = problem

        # If we have reached a satisfactory solution, stop
        if(fitness(problem) == PROBLEM_SIZE):
            break

    return best_solution, number_of_restarts

In [153]:
problem = np.array([False for _ in range(PROBLEM_SIZE)])
solved_problem_rm, iterations_rm = random_mutation_hc(problem, 10000)
solved_problem_sa, iterations_sa = steepest_ascend_hc(problem, 10000, 15)
solved_problem_rr, iterations_rr = random_restart_hc(6500, 10)

print("####Random Mutation Hill Climbing####")
print("original problem fitness:\t", fitness(problem))
print("solved problem fitness:\t\t", fitness(solved_problem_rm))
print("iterations:\t\t\t", iterations_rm)
print("---------------------------------------\n")

print("####Steepest Ascend Hill Climbing####")
print("original problem fitness:\t", fitness(problem))
print("solved problem fitness:\t\t", fitness(solved_problem_sa))
print("iterations:\t\t\t", iterations_sa)
print("---------------------------------------\n")

print("####Random Restart Hill Climbing####")
print("original problem fitness:\t", fitness(problem))
print("solved problem fitness:\t\t", fitness(solved_problem_rr))
print("restarts:\t\t\t", iterations_rr)
print("---------------------------------------\n")

####Random Mutation Hill Climbing####
original problem fitness:	 0
solved problem fitness:		 1000
iterations:			 6078
---------------------------------------

####Steepest Ascend Hill Climbing####
original problem fitness:	 0
solved problem fitness:		 1000
iterations:			 1328
---------------------------------------

####Random Restart Hill Climbing####
original problem fitness:	 0
solved problem fitness:		 1000
restarts:			 2
---------------------------------------



### Simulated-Annealing

This technique is based on performing exploration, and then exploitation, based on an hyperparameter, the temperature, that allows to accept a worse candidate solution with a certain probability.
This is useful to escape from local minima

The probability of accepting a new candidate solution is defined by the `Boltzmann distribution`:
- $S$ = current solution
- $R$ = new candidate solution
- $t$ = current temperature
- $c$ = cooling rate $\in [0,1]$ that defines how fast the temperature decreases

$P(R|S,t) = e^{\frac{Quality(R)-Quality(S)}{t}}$

In [248]:
def simulated_annealing(problem, temperature, cooling_rate, max_iterations):
    number_iterations = 0

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1
        actual_state = deepcopy(problem)
        next_state = tweak(problem)

        if(fitness(next_state) > fitness(actual_state)):
            problem = next_state

            # If we have reached a satisfactory solution, stop
            if(fitness(problem) == PROBLEM_SIZE):
                break
        else:
            # Calculate the probability of accepting the worse solution
            probability = np.exp((fitness(next_state) - fitness(actual_state)) / temperature)

            if(random() < probability):
                problem = next_state

        # Cool the temperature
        temperature *= cooling_rate

        # If the temperature is too low, stop
        if(temperature < 1e-15):
            break

    return problem, number_iterations


In [253]:
solved_problem_ann, iterations_ann = simulated_annealing(problem, 100, 0.999, 10000)

print("####Simulated Annealing####")
print("original problem fitness:\t", fitness(problem))
print("solved problem fitness:\t\t", fitness(solved_problem_ann))
print("iterations:\t\t\t", iterations_ann)
print("---------------------------------------\n")


####Simulated Annealing####
original problem fitness:	 0
solved problem fitness:		 1000
iterations:			 9446
---------------------------------------



### Tabu search
Tabu search is a popular metaheuristic that guides a local search procedure to explore the search space beyond the local optimum.

> 🙅🏻‍♀️ It is based on the idea of `forbidding` the algorithm to visit the same state twice, i.e. it keeps track of the visited states in a `tabu list` and avoids them for a certain number of iterations.

It is particularly effective in solving *combinatorial optimization problems*, where the goal is to find the best solution from a finite set of possibilities. But really only works well when the search space is discrete and finite.
- ⚠️ If the search space is continuous or infinite, only in truly and exceptionally rare cases will the algorithm ever visit the same state twice! In this case, we should check if the current solution is *sufficiently similar* to a previously visited one, and then decide whether to forbid it or not.

In [267]:
def tabu_search(problem, max_iterations):
    number_iterations = 0
    tabu_list = []

    # Loop until we reach the maximum number of iterations
    for _ in range(max_iterations):
        number_iterations += 1
        actual_state = deepcopy(problem)
        next_state = tweak(problem)

        if(fitness(next_state) > fitness(actual_state)):
            problem = next_state

            # If we have reached a satisfactory solution, stop
            if(fitness(problem) == PROBLEM_SIZE):
                break
        else:
            # If the next state is in the tabu list, skip it
            if(fitness(next_state) in tabu_list):
                continue

            problem = next_state

        # Add the actual state to the tabu list
        tabu_list.append(fitness(actual_state))
    return problem, number_iterations

In [269]:
solved_problem_tabu, iterations_tabu = tabu_search(problem, 10000)

print("####Tabu Search####")
print("original problem fitness:\t", fitness(problem))
print("solved problem fitness:\t\t", fitness(solved_problem_tabu))
print("iterations:\t\t\t", iterations_tabu)
print("---------------------------------------\n")

####Tabu Search####
original problem fitness:	 0
solved problem fitness:		 1000
iterations:			 8602
---------------------------------------



### Iterated local search
Iterated local search is a metaheuristic that combines a local search procedure with a perturbation mechanism to escape local optima, and it is essentially a smarter version of random-restart hill climbing.

> It tries to stochasticly hill climb in the space of local optima. To do so, it uses a `home base` solution, which is the best solution found so far, and a `perturbation operator` perform a *just large enough jump* to escape this local optimum and land in a different region of the search space. ⛰️<--🏃🏻‍♂️--🏡

1. Hill climbing is used iteratively for a maximum number of iterations to find a local optimum 
2. If the new local optimum is better than the current home base, it becomes the new home base
3. The perturbation operator is designed to be substantial enough to likely jump to a new region of the solution space